In [ ]:
# Install required dependencies
!pip install torch torchvision opencv-python matplotlib

import torch
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import display, Image

# Load pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

def detect_objects(image_path):
    """Detect objects in an image, draw bounding boxes, and display the result."""

    # Read and convert image to RGB
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run object detection
    results = model(image_rgb)
    detections = results.pandas().xyxy[0]  # Convert results to DataFrame

    # Define bounding box colors
    colors = {
        'person': (0, 255, 0),  # Green
        'car': (255, 0, 0),     # Blue
        'dog': (0, 0, 255)      # Red
    }

    # Filter for desired classes only
    target_classes = ['person', 'car', 'dog']
    filtered_detections = detections[detections['name'].isin(target_classes)]

    # Draw bounding boxes and labels
    for _, det in filtered_detections.iterrows():
        xmin, ymin, xmax, ymax = int(det['xmin']), int(det['ymin']), int(det['xmax']), int(det['ymax'])
        class_name = det['name']
        confidence = det['confidence']
        color = colors.get(class_name, (0, 255, 0))

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)
        label = f"{class_name} {confidence:.2f}"

        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(image, (xmin, ymin - text_height - 10), (xmin + text_width, ymin), color, -1)
        cv2.putText(image, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Save and display processed image
    output_path = 'processed_image.jpg'
    cv2.imwrite(output_path, image)
    display(Image(filename=output_path))
    print(f"Processed image saved as: {output_path}")

# Upload and process image
uploaded = files.upload()
if uploaded:
    filename = next(iter(uploaded))
    detect_objects(filename)
else:
    print("No image uploaded.")
